

### Static Maps

Over the past few weeks, we've been familiarizing ourselves with plotting basic static maps using [`geopandas.GeoDataFrame.plot()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html), as seen in lessons 2, 3, and 4. We've also learned that `geopandas.GeoDataFrame.plot()` leverages the [`matplotlib.pyplot`](https://matplotlib.org/stable/tutorials/introductory/pyplot.html) library, meaning most of `matplotlib`'s arguments and options can be utilized within `geopandas`.

To refresh our understanding and skills in plotting maps, let's tackle a practical exercise: creating a static accessibility map of the Stockholm metropolitan area. This map will incorporate three layers—showing the area, roads, and metro lines—overlayed onto each other. **It's crucial to remember that all input datasets must be in the same coordinate system for accurate representation.**

#### Step-by-Step Guide:

1. **Verify Coordinate Systems**: Ensure that your datasets (Stockholm area, roads, and metro lines) are in the same coordinate system.
2. **Load Datasets**: Use [`geopandas`](https://geopandas.org/) to load each dataset into a GeoDataFrame.
3. **Plotting**: Begin by plotting the Stockholm area as your base layer. Then, overlay the roads and metro lines by using the `.plot()` method on each GeoDataFrame, adjusting the `ax` parameter to ensure they are correctly layered.
4. **Customization**: Utilize [`matplotlib`](https://matplotlib.org/)'s styling options to enhance your map's readability and visual appeal. This could involve adjusting colors, setting line widths, and adding a legend.
5. **Add Final Touches**: Don't forget to include a title, labels, and a legend for clarity and informatial analysis.g!


Happy Mapping!


### Data

We will use three different data sets:

- An administrative deliniation of Sweden, [***DeSO**](https://www.scb.se/hitta-statistik/regional-statistik-och-kartor/regionala-indelningar/deso---demografiska-statistikomraden/): DeSO divides Sweden into 5,984 areas which at the start have between 700 and 2,700 inhabitants. The division takes into account the geographical conditions so that the boundaries, as far as possible, follow, for example, streets, waterways and railways. Important building blocks used to create DeSO are urban areas and electoral districts. Here we use the WFS services of the Swedish Statistical Agency (https://www.scb.se/) to load the layer over Stockholm, Sweden.
  
- A layer of urban areas/small towns [**Tätorter**](https://www.scb.se/vara-tjanster/oppna-data/oppna-geodata/tatorter/) for 1980, again extracted from the web services of the statistical agency. In Sweden, there are more than 2,000 urban areas. It's everything from places with at least 200 inhabitants, to big cities like Stockholm, Gothenburg and Malmö. Over 9 million people or 88 percent of the Swedish population live in an urban area.

In [ ]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [ ]:
import geopandas
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Define the base URL for the WFS request
WFS_BASE_URL = (
    "http://geodata.scb.se/geoserver/stat/wfs"
    "?service=wfs"
    "&version=1.1.0"
    "&request=GetFeature"
    "&srsName=EPSG:3006"
    "&typeName={layer:s}"
)

# Specify the layer you're interested in
# For this example, we use "DeSO.2018" 

# Fetch the data using GeoPandas
deso = geopandas.read_file(
    WFS_BASE_URL.format(layer="DeSO.2018")
).set_crs("EPSG:3006")

# Display the first few rows of the data
print(deso.head())

#Similar process for the urban layer
tatorter_1980 = geopandas.read_file(
    WFS_BASE_URL.format(layer= "stat:Tatorter.1980")
).set_crs("EPSG:3006")

# Display the first few rows of the data
print(tatorter_1980.head())

Remember that different geo-data frames need to be in same coordinate system before plotting them in the same map. geopandas.GeoDataFrame.plot() does not reproject data automatically.

You can always check it with a simple assert statement.

In [ ]:
assert tatorter_1980.crs == deso.crs, "Input data sets’ CRS differs"


If multiple data sets do not share a common CRS, first, figure out which CRS they have assigned (if any!), then transform the data into a common reference system:

In [ ]:
tatorter_1980.crs

In [ ]:
deso = deso.to_crs(tatorter_1980.crs)
tatorter_1980 =tatorter_1980.to_crs(deso.crs)

Let's complete the next steps:
-  Visualise the 2 layers in a map using the geopandas.GeoDataFrame.plot() method 

- style them accordingly

Remember the following options that can be passed to `plot()`:

 While we have only 2 polygon layers in this case, remember that 
- **Style the polygon layer:**
  - Define a classification scheme using the `scheme` parameter.
  - Change the colour map using [cmap](https://matplotlib.org/stable/tutorials/colors/colormaps.html).
  - Control the layer’s transparency with the `alpha` parameter (`0` is fully transparent, `1` is fully opaque).

- **Style the line layers:**
  - Adjust the line colour using [**colour**](https://matplotlib.org/stable/api/colors_api.html).
  - Change the `linewidth`, as needed.
 
- **Style the point layers:**
  - Adjust the point size using the `markersize` parameter to control the visual prominence of each point on the map.
  - Change the point color by specifying the `color` parameter. This allows for uniform coloring or can be based on an attribute using a color map. See [**color options**](https://matplotlib.org/stable/api/colors_api.html) for more details.
  - Utilize the `marker` parameter to change the shape of the points. Matplotlib supports a variety of marker styles, which can be used to differentiate types of locations or data points visuallyerns


The  canlayers have different e. In this case, the layers' extent is for the whole of Sweden but it won't always be like that.area). Use the axes (`ax`) methods `set_xlim()` and `set_ylim()` to set the horizontal and vertical extents of the map, for example, to a GeoDataFrame’s `total_bounds`.


In [ ]:

# Plotting the administrative units ('deso') with visible borders
base = deso.plot(
    figsize=(12, 8),
    linewidth=0.05,  # Increase linewidth for visibility
    edgecolor='black',  # Set edgecolor to black (or any contrasting color) for visibility
    alpha=0.6,  # Maintain some transparency
    color='white'  # Keep the base layer in white
)

# Overlaying the population values ('tatorter_1980') on the same plot
tatorter_1980.plot(
    ax=base,  # Ensure overlaying on the base plot
    column='bef',  # 'bef' is the column name for population values
    scheme='quantiles',
    cmap='Spectral',
    linewidth=0,  # Set to 0 or a small value to avoid cluttering the map 
    legend=False,  # Enable legend for population quantiles
    alpha=0.75  # Keep the overlay semi-transparent to see the base layer beneath
)

# Remove axis display for a cleaner look
base.set_axis_off()


### Map Legends

To include a legend in your map, utilize the `legend=True` parameter during plotting.

- For figures **without** a classification scheme, the legend will display as a color gradient bar. This legend is an instance of [`matplotlib.pyplot.colorbar.Colorbar`](https://matplotlib.org/stable/api/colorbar_api.html).
- You can pass various arguments to customize the legend through `legend_kwds`. These arguments are directly passed to the colorbar instance.  Discover more about customizing your color bar by reviewing the [Colorbar documentation](https://matplotlib.org/stable/api/colorbar_api.html).
  
For example, to set the title of the legend, use the `label` property within `legend_kwds`:

```python
your_geodataframe.plot(
    ...,
    legend=True,
    legend_kwds={'label': 'Your Legend Title Here'}
)


In [ ]:
# Plotting the administrative units ('deso') with visible borders
base = deso.plot(
    figsize=(12, 8),
    linewidth=0.05,  # Increase linewidth for visibility
    edgecolor='black',  # Set edgecolor to black (or any contrasting color) for visibility
    alpha=0.6,  # Maintain some transparency
    color='white'  # Keep the base layer in white
)

# Overlaying the population values ('tatorter_1980') on the same plot
tatorter_1980.plot(
    ax=base,  # Ensure overlaying on the base plot
    column='bef',  # 'bef' is the column name for population values
    cmap='Spectral_r',
    linewidth=0,  # Set to 0 or a small value to avoid cluttering the map 
    legend=True,  # Enable legend 
    legend_kwds={"label": "Urban Population in 1980"},
    alpha=0.75  # Keep the overlay semi-transparent to see the base layer beneath
)

# Remove axis display for a cleaner look
base.set_axis_off()


### Enhancing Your Map Legends

For figures that utilize a classification scheme, `plot()` generates a [`matplotlib.legend.Legend`](https://matplotlib.org/stable/api/legend_api.html). While you can still use `legend_kwds` to customize your legend, the parameters slightly differ from those used with color bars. For instance, to set the legend title, you should use `title` instead of `label`:

```python
your_geodataframe.plot(
    ...,
    legend=True,
    legend_kwds={'title': 'Your Legend Title Here'}
)


In [ ]:
# First, plot the 'deso' DataFrame to establish the base layer
base = deso.plot(
    figsize=(12, 8),
    color='lightgrey',  # A neutral background color
    linewidth=0.02,  # Visible borders for administrative units
    edgecolor='black',  # Border color to enhance visibility
    alpha=0.4  # Slightly transparent to allow overlay visibility
)

# Next, overlay the 'tatorter_1980' DataFrame on the same Axes object
tatorter_1980.plot(
    ax=base,  # Plot on the base layer
    column="bef",  # Column for urban population in 1980
    cmap="Spectral_r",  # Use the inverted color map for visual distinction
    scheme='Quantiles',  # Classification scheme for the data
    k=10,  # Number of classes
    legend=True,  # Enable legend
    legend_kwds={
        'title': "Urban Population in 1980",  # Title for the legend
        'loc': 'upper left',  # Position of the legend
    },
    linewidth=0,  # No border lines
    alpha=0.8,  # Slightly transparent
)

base.set_axis_off()  # Hide the axis



### Customizing Your Legends

Dive deeper into customizing your map legends by exploring the [`matplotlib.pyplot.legend.Legend` documentation](https://matplotlib.org/stable/api/legend_api.html). The `legend_kwds` dictionary allows you to pass a wide array of parameters directly to the legend, offering a plethora of customization options.

Want to experiment with the layout of your legend? Consider this: to spread the legend across two columns, you can use the `ncol` keyword within `legend_kwds`. Here's how:

```python
your_geodataframe.plot(
    ...,
    legend=True,
    legend_kwds={'title': 'Your Legend Title', 'ncol': 2}
)


In [ ]:
# First, plot the 'deso' DataFrame to establish the base layer
base = deso.plot(
    figsize=(12, 8),
    color='lightgrey',  # A neutral background color
    linewidth=0.02,  # Visible borders for administrative units
    edgecolor='black',  # Border color to enhance visibility
    alpha=0.4  # Slightly transparent to allow overlay visibility
)

# Next, overlay the 'tatorter_1980' DataFrame on the same Axes object
tatorter_1980.plot(
    ax=base,  # Plot on the base layer
    column="bef",  # Column for urban population in 1980
    cmap="Spectral_r",  # Use the inverted color map for visual distinction
    scheme='Quantiles',  # Classification scheme for the data
    k=10,  # Number of classes
    legend=True,  # Enable legend
    legend_kwds={
        'title': "Urban Population in 1980",  # Title for the legend
        'loc': 'upper left',
        'ncol': 2# Position of the legend
    },
    linewidth=0,  # No border lines
    alpha=0.8,  # Slightly transparent
)

base.set_axis_off()  # Hide the axis



### Adding a base map

For better orientation, it is often helpful to add a base map to a map plot. A base map, for instance, from map providers such as [OpenStreetMap](https://www.openstreetmap.org/#map=5/62.994/17.637) or [Stamen](https://maps.stamen.com/#terrain/12/37.7706/-122.3782), adds streets, place names, and other contextual information.

The Python package contextily takes care of downloading the necessary map tiles and rendering them in a geopandas plot.

Map tiles from online map providers are typically in Web Mercator projection (EPSG:3857, https://epsg.io/3857). It is generally advisable to transform all other layers to EPSG:3857, too.

In [ ]:
deso=deso.to_crs("EPSG:3857")
tatorter_1980=tatorter_1980.to_crs("EPSG:3857")

In [ ]:
deso.crs

To add a base map to an existing plot, use the [contextily.add_basemap()](https://contextily.readthedocs.io/en/latest/intro_guide.html) function, and supply the plot’s ax object obtained in an earlier step. By default, contextily uses the Stamen Terrain as a base map, but there are many other online maps to choose from. Any of the other contextily.providers (see link above) can be passed as a source to add_basemap(). For instance, use OpenStreetMap in its default Mapnik style:

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd
# Step 4: Explicitly create a figure and axes
fig, ax = plt.subplots(figsize=(12, 8))

# Plot 'deso' as base layer
deso.plot(
    ax=ax,
    color='lightgrey',
    linewidth=0.02,
    edgecolor='black',
    alpha=0.7,
)

# Overlay 'tatorter_1980' with quantiles classification
tatorter_1980.plot(
    ax=ax,
    column="bef",
    cmap="Spectral_r",
    scheme='Quantiles',
    k=5,
    legend=True,
    legend_kwds={'title': "Urban Population in 1980", 'loc': 'upper right'},
    linewidth=0,
    alpha=0.8,
)

# Add the basemap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Optional: Adjust the plot limits to the data extent if necessary
ax.set_xlim(tatorter_1980.total_bounds[[0, 2]])
ax.set_ylim(tatorter_1980.total_bounds[[1, 3]])

# Show the plot
plt.show()

In [ ]:

# Step 4: Explicitly create a figure and axes
fig, ax = plt.subplots(figsize=(12, 8))


# Overlay 'tatorter_1980' with quantiles classification
tatorter_1980.plot(
    ax=ax,
    column="bef",
    cmap="Spectral_r",
    scheme='Quantiles',
    k=5,
    legend=True,
    legend_kwds={'title': "Urban Population in 1980", 'loc': 'upper right'},
    linewidth=0,
    alpha=0.8,
)

# Add the basemap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Optional: Adjust the plot limits to the data extent if necessary
ax.set_xlim(tatorter_1980.total_bounds[[0, 2]])
ax.set_ylim(tatorter_1980.total_bounds[[1, 3]])

# Show the plot
plt.show()

In this zoom level, the benefits from using OpenStreetMap (such as place names) do not live to their full potential. Let’s look at a subset  of Sweden. Let's filter the dataset to visualize only the regions that contain the top 1% most populated urban areas.

In [ ]:

import numpy as np


# Step 1: Filter to include only the top 95% of population values
# Calculate the 5th percentile value as the cutoff for the top 99%
cutoff = tatorter_1980['bef'].quantile(0.995)

# Filter the GeoDataFrame
top_95_percent = tatorter_1980[tatorter_1980['bef'] > cutoff]

# Plot this filtered data
ax = top_95_percent.plot(
    figsize=(12, 8),
    column="bef",
    scheme="Quantiles",
    k=7,
    cmap="Spectral_r",
    linewidth=0,
    alpha=0.8,
    legend=True,
    legend_kwds={"title": "Urban Population in 1980 (Top 95%)"}
)

# Step 2: Adjust the plot limits to the filtered data extents


# This effectively zooms in on the area of interest
minx, miny, maxx, maxy = top_95_percent.total_bounds
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

# Add a basemap with adjusted zoom to fit the extents of the filtered data
ctx.add_basemap(
    ax,
    source=ctx.providers.OpenStreetMap.Mapnik,
    # The zoom level is automatically adjusted to fit the extents
)

plt.show()


In [ ]:


# Filter to include only the top 99% of population values
# Plot this filtered data
ax =tatorter_1980[tatorter_1980['bef'] > tatorter_1980['bef'].quantile(0.999)].plot(
    figsize=(12, 8),
    column="bef",
    scheme="Quantiles",
    k=7,
    cmap="Spectral_r",
    linewidth=0,
    alpha=0.8,
    legend=True,
    legend_kwds={"title": "Urban Population in 1980 (Top 99%)",
                "loc": "upper left"}
)


# Add a basemap with adjusted zoom to fit the extents of the filtered data
ctx.add_basemap(
    ax,
    source=ctx.providers.OpenStreetMap.Mapnik,
    # The zoom level is automatically adjusted to fit the extents
)

plt.show()

In [ ]:


# Filter the 'tatorter_1980' GeoDataFrame for rows where 'kommunnamn' is 'Stockholm'
stockholm_data = tatorter_1980[tatorter_1980['kommunnamn'] == 'Stockholm']


# Initialize a plot with matplotlib
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the filtered data
stockholm_data.plot(
    ax=ax,
    color="red",  # Use a single color to represent Stockholm
    edgecolor="black",
    linewidth=0.005,
    alpha=0.2,
)

# Add a basemap with adjusted zoom to fit the extents of the filtered data
ctx.add_basemap(
    ax,
    source=ctx.providers.OpenStreetMap.Mapnik,
)

plt.show()


Finally, we can modify the attribution (copyright notice) displayed in the bottom left of the map plot. Note that you should always respect the map providers’ terms of use, which typically include a data source attribution (contextily’s defaults take care of this). We can and should, however, add a data source for any layer we added, such as the travel time matrix data set:

In [ ]:

# Filter the 'tatorter_1980' GeoDataFrame for rows where 'kommunnamn' is 'Stockholm'
stockholm_data = tatorter_1980[tatorter_1980['kommunnamn'] == 'Stockholm']


# Initialize a plot with matplotlib
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the filtered data
stockholm_data.plot(
    ax=ax,
    color="red",  # Use a single color to represent Stockholm
    edgecolor="black",
    linewidth=0.005,
    alpha=0.2,
)

# Add a basemap with adjusted zoom to fit the extents of the filtered data
ctx.add_basemap(
    ax,
    source=ctx.providers.OpenStreetMap.Mapnik,
    attribution=(
        "Urban areas derived from SCB.SE, "
        "map data (c) OpenStreetMap contributors"
    )
)
plt.show()
